In [1]:
import pandas as pd

In [3]:
from datetime import datetime,timedelta

In [24]:
# Step 1: Load Data from CSV
center_data = pd.read_csv(r"D:\Data Analyist\Mysql\class\Projects\NIA Project\centers.csv",sep = ",")
candidates_data = pd.read_csv(r"D:\Data Analyist\Mysql\class\Projects\NIA Project\candidate.csv",sep = ",")

In [25]:
candidates_df = candidates_data
centers_df = center_data

In [26]:
import math

In [27]:
# Step 2: Calculate Required Exam Days
def calculate_required_days(candidates_df,centers_df) :
    total_candidates = len(candidates_df)
    total_seats_per_day = centers_df["Seats_Per_Day"].sum()
    required_days = math.ceil(total_candidates/total_seats_per_day)
    print(f"\n📌 Total Candidates  = {total_candidates}")
    print(f"\n📌 Total Seats Per Day = {total_seats_per_day}")
    print(f"\n📌 Required Days = {required_days}")
    return required_days


In [30]:
# Call the function
required_days = calculate_required_days(candidates_df,centers_df)


📌 Total Candidates  = 150

📌 Total Seats Per Day = 118

📌 Required Days = 2


In [42]:
from datetime import datetime

def get_exam_dates(required_days):
    exam_dates = []
    print(f"\nPlease enter {required_days} exam dates (format DD-MM-YYYY):")
    
    for i in range(required_days):
        while True:
            date_str = input(f"Enter exam date {i+1}: ")
            try:
                # Validate date format
                dt = datetime.strptime(date_str, "%d-%m-%Y")
                exam_dates.append(dt.strftime("%Y-%m-%d"))
                break
            except ValueError:
                print("Invalid format! Please enter date in DD-MM-YYYY format.")
    
    return exam_dates  # <-- return at the end, not inside except


In [43]:
exam_dates = get_exam_dates(required_days)


Please enter 2 exam dates (format DD-MM-YYYY):


Enter exam date 1:  02-06-2025
Enter exam date 2:  03-06-2025


In [48]:
# Strip spaces and make all names consistent
for col in ["Preference_1","Preference_2","Preference_3","Preference_4"]:
    candidates_df[col] = candidates_df[col].str.strip()

centers_df['Location'] = centers_df['Location'].str.strip()


In [54]:
def allocate_centers_debug(candidates_df, centers_df, exam_dates):
    seat_tracker = {row["Location"]: {d: 0 for d in exam_dates} for _, row in centers_df.iterrows()}
    candidates_df["Allocated_Center"] = None
    candidates_df["Allocated_Date"] = None

    for idx, candidate in candidates_df.iterrows():
        allocated = False
        for pref in ["Preference_1","Preference_2","Preference_3","Preference_4"]:
            pref_loc = candidate[pref]
            if pref_loc not in centers_df["Location"].values:
                # Debug: preference not found
                print(f"Candidate {candidate['Candidate_ID']} preference '{pref_loc}' not in centers")
                continue

            seats_per_day = centers_df.loc[centers_df["Location"] == pref_loc, "Seats_Per_Day"].values[0]

            for d in exam_dates:
                if seat_tracker[pref_loc][d] < seats_per_day:
                    seat_tracker[pref_loc][d] += 1
                    candidates_df.at[idx, "Allocated_Center"] = pref_loc
                    candidates_df.at[idx, "Allocated_Date"] = d
                    allocated = True
                    break
            if allocated:
                break

        if not allocated:
            candidates_df.at[idx, "Allocated_Center"] = "Not Allocated"
            candidates_df.at[idx, "Allocated_Date"] = "N/A"

    return candidates_df, seat_tracker


In [74]:
candidates_df, seat_tracker = allocate_centers_debug(candidates_df, centers_df, exam_dates)
print(candidates_df.head(5))

  Candidate_ID         Name Preference_1 Preference_2 Preference_3  \
0         U001  Candidate_1        Delhi       Mumbai         Pune   
1         U002  Candidate_2        Delhi      Kolkata       Nagpur   
2         U003  Candidate_3       Mumbai        Delhi        Patna   
3         U004  Candidate_4      Chennai    Hyderabad        Surat   
4         U005  Candidate_5      Lucknow        Delhi       Jaipur   

  Preference_4 Allocated_Center Allocated_Date  
0       Jaipur            Delhi     2025-09-15  
1      Chennai            Delhi     2025-09-15  
2       Bhopal           Mumbai     2025-09-15  
3       Nagpur          Chennai     2025-09-15  
4        Patna          Lucknow     2025-09-15  


In [75]:
# Step 5: Save Output to CSV
# -----------------------------
def save_results(candidates_df, output_file="final_allocation.csv"):
    candidates_df.to_csv(output_file, index=False)
    print(f"\n✅ Final allocation saved to {output_file}")

In [77]:
candidates_df

,Candidate_ID,Name,Preference_1,Preference_2,Preference_3,Preference_4,Allocated_Center,Allocated_Date
0,U001,Candidate_1,Delhi,Mumbai,Pune,Jaipur,Delhi,2025-09-15
1,U002,Candidate_2,Delhi,Kolkata,Nagpur,Chennai,Delhi,2025-09-15
2,U003,Candidate_3,Mumbai,Delhi,Patna,Bhopal,Mumbai,2025-09-15
3,U004,Candidate_4,Chennai,Hyderabad,Surat,Nagpur,Chennai,2025-09-15
4,U005,Candidate_5,Lucknow,Delhi,Jaipur,Patna,Lucknow,2025-09-15
...,...,...,...,...,...,...,...,...
145,U146,Candidate_146,Delhi,Hyderabad,Pune,Lucknow,Hyderabad,2025-09-16
146,U147,Candidate_147,Bangalore,Mumbai,Noida,Chennai,Bangalore,2025-09-16
147,U148,Candidate_148,Indore,Delhi,Jaipur,Ahmedabad,Indore,2025-09-15
148,U149,Candidate_149,Surat,Lucknow,Bhopal,Patna,Surat,2025-09-16


In [79]:
# save file to donload 
candidates_df.to_csv("candidates_df.csv", index=False)

In [95]:
# Step 5: Generate Seat Availability Report 

def seat_availability_report(centers_df, seat_tracker, exam_dates):
    """
    Generates a detailed seat availability report per center per exam date.
    
    """
    rows = []
    
    for idx, row in centers_df.iterrows():
        center = row["Location"]
        seats_per_day = row["Seats_Per_Day"]
        
        for d in exam_dates:
            allocated = seat_tracker.get(center, {}).get(d, 0)
            remaining = max(seats_per_day - allocated, 0)
            
            rows.append({
                "Center": center,
                "Date": d,
                "Total_Seats": seats_per_day,
                "Allocated": allocated,
                "Remaining": remaining
            })
    
    report_df = pd.DataFrame(rows)
    
    # Optional: Sort by Center and Date for readability
    report_df = report_df.sort_values(by=["Center", "Date"]).reset_index(drop=True)
    
    return report_df


In [96]:
report_df = seat_availability_report(centers_df, seat_tracker, exam_dates)

In [97]:
report_df

,Center,Date,Total_Seats,Allocated,Remaining
0,Ahmedabad,2025-09-15,6,4,2
1,Ahmedabad,2025-09-16,6,0,6
2,Ahmedabad,2025-09-17,6,0,6
3,Bangalore,2025-09-15,6,6,0
4,Bangalore,2025-09-16,6,2,4
5,Bangalore,2025-09-17,6,0,6
6,Bhopal,2025-09-15,5,5,0
7,Bhopal,2025-09-16,5,3,2
8,Bhopal,2025-09-17,5,0,5
9,Chandigarh,2025-09-15,5,5,0


In [98]:
# Step 6: Generate Not Allocated Candidates Report
# ------------------------------------------------
def not_allocated_report(candidates_df):
    """
    Generates a report of candidates who were not allocated to any center.
    
    """
    # Filter candidates where allocation failed
    not_allocated_df = candidates_df[candidates_df["Allocated_Center"] == "Not Allocated"].copy()
    
    # Optional: select relevant columns
    not_allocated_df = not_allocated_df[["Candidate_ID", "Name", 
                                         "Preference_1", "Preference_2", 
                                         "Preference_3", "Preference_4"]]
    
    return not_allocated_df

In [99]:
not_allocated_df = not_allocated_report(candidates_df)

In [100]:
not_allocated_df

,Candidate_ID,Name,Preference_1,Preference_2,Preference_3,Preference_4


In [101]:
# Step 8: Generate Duplicate Submission Report
# --------------------------------------------
def duplicate_submission_report(candidates_df):
    """
    Generates a report of candidates who submitted the form multiple times.
    
    """
    # Find duplicates based on Candidate_ID
    duplicates_df = candidates_df[candidates_df.duplicated(subset="Candidate_ID", keep=False)].copy()
    
    # Sort by Candidate_ID for readability
    duplicates_df = duplicates_df.sort_values(by="Candidate_ID").reset_index(drop=True)
    
    # Optional: select relevant columns
    duplicates_df = duplicates_df[["Candidate_ID", "Name", 
                                   "Preference_1", "Preference_2", 
                                   "Preference_3", "Preference_4"]]
    
    return duplicates_df


In [102]:
duplicates_df = duplicate_submission_report(candidates_df)

In [103]:
duplicates_df

,Candidate_ID,Name,Preference_1,Preference_2,Preference_3,Preference_4


In [104]:
## Project END 